In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
import pandas as pd
from tqdm import tqdm, tqdm_notebook
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix

In [ ]:
n_s = 50
n_f = 15

In [ ]:
pwd

'/content'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/datasets_roorke/

/content/drive/My Drive/datasets_roorke


In [ ]:
!nvidia-smi

Tue Feb  4 16:24:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
d_m = pd.read_csv('GWO_selectedFeatures_MultiTasks_STEW_TrainTest.csv',names=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])
d_n = pd.read_csv('GWO_SelectedFeatures_STEWTrainTest.csv',names=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

In [ ]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          26126        1275       20970           0        3879       24767
Swap:             0           0           0


In [ ]:
d_n.shape

(1048044, 16)

In [ ]:
#data_m = np.array([(d_m.values[:,i]-np.min(d_m.values[:,i]))/(np.max(d_m.values[:,i])-np.min(d_m.values[:,i])) for i in tqdm(range(d_m.values.shape[1]-1))]).T
#data_n = np.array([(d_n.values[:,i]-np.min(d_n.values[:,i]))/(np.max(d_n.values[:,i])-np.min(d_n.values[:,i])) for i in tqdm(range(d_n.values.shape[1]-1))]).T

label_m = [d_m.values[:,-1]]
label_n = d_n.values[:,-1]

100%|██████████| 15/15 [00:01<00:00, 11.09it/s]


In [ ]:
def one_hot(d):
  l = []
  for i in d:
    if i == 0:
      l.extend(np.asarray([1,0]))
    if i==1:
      l.extend(np.asarray([0,1]))
  return np.asarray(l)

In [ ]:
label_n = one_hot(d_n.values[:,-1])

In [ ]:
label_n = label_n.reshape(d_n.shape[0],2)

In [ ]:
data_m.shape

(1048214, 15)

In [ ]:
def seq_generator1(d,l,length):
  seq_data = []
  seq_label = []
  d.shape
  i = 600000
  for j in tqdm_notebook(range(600000+length,d.shape[0])):
    if i==800000:
      break
    seq_data.extend(np.asarray([np.asarray(d[i:j,:])]))
    seq_label.extend(np.asarray([np.asarray(k) for k in l[i:j] ]))
    i += 1
    
  return np.asarray(seq_data), np.asarray(seq_label)

In [ ]:
with tf.device('/GPU:0'):
  #data_mul, label_mul = seq_generator1(d=data_m, l=label_m, length=100)
  data_no2, label_no2 = seq_generator1(d=d_n.values[:,:-1], l= label_n, length=100)

In [ ]:
del data_n

In [ ]:
#data_no  = np.asarray(data_no)
label_no2.shape

(10000000, 2)

In [ ]:
label_no1=label_no1.reshape(600000,100,2)

In [ ]:
np.unique(trainy)

NameError: ignored

In [ ]:
label_no1=np.asarray([np.asarray([[]])])

In [ ]:
np.unique(label_m),np.unique(label_n) 

In [ ]:
trainx, trainy  = data_no1[:550000],label_no1[:550000]

In [ ]:
trainx.max()

111.2259442

In [ ]:
n_s = trainx.shape[1]
n_f = trainx.shape[2]

In [ ]:
encoder_input = tf.keras.layers.Input(shape=(n_s,n_f,))
x = tf.keras.layers.LSTM(16,activation="tanh",return_sequences=True)(encoder_input)
x = tf.keras.layers.LSTM(32,activation='tanh',return_sequences=True)(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.LSTM(128,activation='tanh',return_sequences=True)(x)

encoder_output = tf.keras.layers.LSTM(256,activation='tanh',return_sequences=False)(x)

decoder_input = tf.keras.layers.RepeatVector(n_s)(encoder_output)

x = tf.keras.layers.LSTM(128,activation='tanh',return_sequences=True)(decoder_input)
x = tf.keras.layers.LSTM(64,activation='tanh',return_sequences=True)(x)
x = tf.keras.layers.Dropout(0.5)(x)
decoder_output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(2, activation='sigmoid'))(x)

model_sae_lstm = Model(inputs=encoder_input, outputs=decoder_output)
model_sae_lstm.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100, 15)]         0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100, 16)           2048      
_________________________________________________________________
lstm_7 (LSTM)                (None, 100, 32)           6272      
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 32)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 100, 128)          82432     
_________________________________________________________________
lstm_9 (LSTM)                (None, 256)               394240    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 100, 256)          0   

In [ ]:
model_sae_lstm.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'],loss=tf.keras.losses.binary_crossentropy)

In [ ]:
history = model_sae_lstm.fit(trainx, trainy,validation_data=(data_no2[:100000],label_no2.reshape(200000,100,2)[:100000]), epochs=100,batch_size=5500, steps_per_epoch=100)
#model_sae_lstm.save_weights('model_sae_lstm_no.h5')

Train on 550000 samples, validate on 100000 samples
Epoch 1/100
550000/550000 [==============================] - 66s 120us/sample - loss: 0.1329 - accuracy: 0.9450 - val_loss: 0.2325 - val_accuracy: 0.9205
Epoch 2/100
550000/550000 [==============================] - 66s 119us/sample - loss: 0.1280 - accuracy: 0.9476 - val_loss: 0.3337 - val_accuracy: 0.9094
Epoch 3/100
550000/550000 [==============================] - 66s 120us/sample - loss: 0.1264 - accuracy: 0.9487 - val_loss: 0.2862 - val_accuracy: 0.9172
Epoch 4/100
550000/550000 [==============================] - 64s 116us/sample - loss: 0.1241 - accuracy: 0.9499 - val_loss: 0.2852 - val_accuracy: 0.9182
Epoch 5/100
550000/550000 [==============================] - 65s 119us/sample - loss: 0.1236 - accuracy: 0.9503 - val_loss: 0.3136 - val_accuracy: 0.9164
Epoch 6/100
550000/550000 [==============================] - 66s 120us/sample - loss: 0.1218 - accuracy: 0.9513 - val_loss: 0.3304 - val_accuracy: 0.9138
Epoch 7/100
550000/55000

KeyboardInterrupt: ignored

In [ ]:
model_sae_lstm.evaluate(data_no2,label_no2.reshape(200000,100,2))

200000/200000 [==============================] - 77s 386us/sample - loss: 0.3145 - accuracy: 0.9066


[0.31447952224034814, 0.9065597]

In [ ]:
model_sae_lstm.save('model_sae_lstm_no50.h5')
model_sae_lstm.save_weights('model_sae_lstm_no_weights50.h5')

In [ ]:
model_sae_lstm.predict(data_no2)

array([[[9.9145377e-01, 8.3688116e-03],
        [9.9844438e-01, 1.5254086e-03],
        [9.9907792e-01, 8.9112035e-04],
        ...,
        [9.9910009e-01, 8.8842184e-04],
        [9.9909997e-01, 8.8849757e-04],
        [9.9909997e-01, 8.8856911e-04]],

       [[9.9146104e-01, 8.3615649e-03],
        [9.9844557e-01, 1.5241578e-03],
        [9.9907887e-01, 8.9019368e-04],
        ...,
        [9.9910128e-01, 8.8728953e-04],
        [9.9910116e-01, 8.8736595e-04],
        [9.9910104e-01, 8.8743918e-04]],

       [[9.9146724e-01, 8.3553325e-03],
        [9.9844664e-01, 1.5230929e-03],
        [9.9907959e-01, 8.8941108e-04],
        ...,
        [9.9910212e-01, 8.8647060e-04],
        [9.9910200e-01, 8.8654796e-04],
        [9.9910200e-01, 8.8662095e-04]],

       ...,

       [[4.8524517e-01, 5.1416487e-01],
        [3.6921260e-01, 6.3770729e-01],
        [3.1948045e-01, 6.9083577e-01],
        ...,
        [4.1680175e-01, 5.8750510e-01],
        [4.1680172e-01, 5.8750516e-01],
        [

In [ ]:
score = model_sae_lstm.evaluate(data_no2[100000:], label_no2.reshape(200000,100,2)[100000:])

100000/100000 [==============================] - 39s 388us/sample - loss: 0.4957 - accuracy: 0.8742


In [ ]:
model = tf.keras.models.load_model("model_sae_lstm_no.h5")
model.evaluate(data_no2[100000:], label_no2.reshape(200000,100,2)[100000:])

100000/100000 [==============================] - 241s 2ms/sample - loss: 0.3587 - accuracy: 0.9007


[0.3586525441920012, 0.9007014]

In [ ]:
yhat = model.predict(data_no2)
predictions = yhat.argmax(axis=-1)


In [ ]:
predictions = yhat.argmax(axis=-1)
predictions.shape

(200000, 100)

In [ ]:
y_true = np.where(label_no2.reshape(200000,100,2)[:,:,0] == 0,1,0)

In [ ]:
cm = confusion_matrix(y_true=y_true.flatten(),y_pred = predictions.flatten())

In [ ]:
tn, fp, fn, tp=cm.ravel()

In [ ]:
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
print(sensitivity, specificity)

0.11870260416666667 0.9902088495575221


In [ ]:
y_pred